<a href="https://colab.research.google.com/github/28YOGESH/proGENTRL/blob/master/PROGENTRL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! pip install progentrl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.6/51.6 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.0/719.0 kB 58.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 67.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 62.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 71.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 21.7 MB/s eta 0:00:00
  Created wheel for pomegranate: filename=pomegranate-0.12.0-cp310-cp310-linux_x86_64.whl size=14204225 sha256=e24d7b91be57593be7ce7810b9

In [4]:
! pip install rdkit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 33.8 MB/s eta 0:00:00


In [5]:
import progentrl as gentrl
import torch
import pandas as pd

from tqdm import tqdm_notebook

import warnings
warnings.simplefilter('ignore')

/usr/local/lib/python3.10/dist-packages/moses/metrics/utils.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  _mcf.append(_pains, sort=True)['smarts'].values]


In [6]:
from pytorch_lightning import Trainer

In [7]:
from moses.metrics import mol_passes_filters, QED, SA, logP
from moses.metrics.utils import get_n_rings, get_mol


def get_num_rings_6(mol):
    r = mol.GetRingInfo()
    return len([x for x in r.AtomRings() if len(x) > 6])


def penalized_logP(mol_or_smiles, masked=False, default=-5):
    mol = get_mol(mol_or_smiles)
    if mol is None:
        return default
    reward = logP(mol) - SA(mol) - get_num_rings_6(mol)
    if masked and not mol_passes_filters(mol):
        return default
    return reward

In [8]:
df = pd.read_csv('DATASET.csv')
df = df[df['SPLIT'] == 'train']
df['plogP'] = df['SMILES'].apply(penalized_logP)
df.to_csv('train_plogp_plogpm.csv', index=None)

In [29]:
A = gentrl.MolecularDataset(sources=[{
          'path':'cb1.csv',
          'smiles': 'canonicalsmiles',
          'prob': 0.5,
          'acvalue' : 'acvalue',
           }], 
        props=['acvalue'])

from torch.utils.data import DataLoader
train_loader = DataLoader(A, batch_size=50, shuffle=True, num_workers=1, drop_last=True)

In [32]:
md = gentrl.MolecularDataset(sources=[
    {'path':'train_plogp_plogpm.csv',
     'smiles': 'SMILES',
     'prob': 0.5,
     'plogP' : 'plogP',
    }], 
    props=['plogP'])

from torch.utils.data import DataLoader
train_loader = DataLoader(md, batch_size=50, shuffle=True, num_workers=1, drop_last=True)

In [33]:
enc = gentrl.RNNEncoder(latent_size=50)
dec = gentrl.DilConvDecoder(latent_input_size=50)
latent_descr = 50 * [('c', 20)]
feature_descr = [('c', 20)]

In [34]:
model = gentrl.gentrlVAE(enc,
                         dec,
                         train_loader,
                         latent_descr,
                         feature_descr)

In [35]:
trainer = Trainer(max_epochs=20)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(model)  # Training the model

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name | Type           | Params
----------------------------------------
0 | enc  | RNNEncoder     | 888 K 
1 | dec  | DilConvDecoder | 596 K 
2 | lp   | LP             | 1.6 M 
----------------------------------------
3.1 M     Trainable params
0         Non-trainable params
3.1 M     Total params
12.473    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

reinit Done from data
reinit Done from data
reinit Done from data


In [18]:
gentrl.save(model, './model')  # Returns true on successful completion

True

In [19]:
model = gentrl.gentrlRL(penalized_logP, 
                        enc,
                        dec,
                        latent_descr,
                        feature_descr,
                       load_model='./model/')

In [24]:
trainer = Trainer(max_epochs=20)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [25]:
trainer.fit(model)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name | Type           | Params
----------------------------------------
0 | enc  | RNNEncoder     | 888 K 
1 | dec  | DilConvDecoder | 596 K 
2 | lp   | LP             | 1.6 M 
----------------------------------------
3.1 M     Trainable params
0         Non-trainable params
3.1 M     Total params
12.473    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Streaming output truncated to the last 5000 lines.
[10:32:10] SMILES Parse Error: extra close parentheses while parsing: CCOC(=O)CCC(=O)Nc1ccc(C(F)(F)F)s2)cc1OC
[10:32:10] SMILES Parse Error: Failed parsing SMILES 'CCOC(=O)CCC(=O)Nc1ccc(C(F)(F)F)s2)cc1OC' for input: 'CCOC(=O)CCC(=O)Nc1ccc(C(F)(F)F)s2)cc1OC'
[10:32:10] SMILES Parse Error: unclosed ring for input: 'COc1ccccc1C(=O)Nc1csC(C)C'
[10:32:10] SMILES Parse Error: extra open parentheses for input: 'Cc1ccc(Cn2c3cn3nc(N(C)C)cc(C(F)(F)F)n3)cc(C(F)3'
[10:32:10] SMILES Parse Error: extra close parentheses while parsing: CC(C)NC(=O)Cc1cccs1)c(C(F)(F)F)cc1
[10:32:10] SMILES Parse Error: Failed parsing SMILES 'CC(C)NC(=O)Cc1cccs1)c(C(F)(F)F)cc1' for input: 'CC(C)NC(=O)Cc1cccs1)c(C(F)(F)F)cc1'
[10:32:10] SMILES Parse Error: unclosed ring for input: 'COc1ccc(NC(=O)c2coc3c(-c5ccccncc3)n3)cc1'
[10:32:10] Can't kekulize mol.  Unkekulized atoms: 1 2 3 5 7
[10:32:10] SMILES Parse Error: syntax error while parsing: Cc1[CCN(CC)S(=O)(=O)Cc1cccc(OC

In [26]:
gentrl.save(model, './model')

True

In [27]:
model(10)

['C#CC(CO)NS(=O)(=O)c1c(Cl)cc(Cl)c1Cl',
 'CCOC(=O)C1CC=NC(=O)c2ccc(Sc3ccccc3)ccn2C)cc1',
 'COc1ccc(-c2csc(NC(=O)c3ccccc3)c2n1',
 'COC(=O)c1sccc1-c1ccsc1C(=O)c1cccs1',
 'COc1ccc(NC(=O)C(C)Sc2ccccc2)cc1',
 'CCOC(=O)c1sc(CC)c(C)c2cc1OCC(=O)OCc2ccccc2)CC1',
 'CCOC(=O)CSc1nc(N)c(C)cc(CC)c2cccs1',
 'CCOC(=O)c1nn(-c2ccccc2)nc1C(C)C',
 'O=C(CSc1ccc(C(F)(F)F)s1)c1ccc2c(c1)OCO2',
 'c1ccc(-c2csc3ncccc3o2)ccc1O']